In [1]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib
import time
from IPython.display import Image

from sklearn import tree
from dtreeviz.trees import *

In [2]:
def highlight_max(data, color='yellow'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)

<p>
  <font size="6">$$b{:}\mathcal{X}^{(m)} {\rightarrow} \mathcal{Y}^{(l)}$$</font> 
</p>


# MARLENA: explaining multi-label black-box decisions

### Multi-label classification

Multi-label tasks are quite common, everytime an instance might be associated with more than one not mutually exclusive labels we have a multi-label problem. 


<center><img style="float: ;" src="./data/gitignoreDATA/presentazione_lipari_MARLENA/carlino_multilabel.png"></center>

<center><font size="6">[Dog, Wig, Guitar]</font></center>



Most of the high-performance algorithms are black-boxes. How can we explain a multi-label black box decision using **MARLENA**?

Let's walk through an example.

### Dataset: *NYS OMH Patient Characteristics Survey (PCS) 2015*
This dataset contains demographic, clinical, social, and insurance characteristics for each client served by the NYS public mental health system during the week of October 19, 2015.

<center><img style="width: 60%; height: 60%" src="./data/gitignoreDATA/presentazione_lipari_MARLENA/Data.png"></center>

https://www.kaggle.com/new-york-state/nys-patient-characteristics-survey-pcs-2015

In [4]:
df.columns

Index(['Age Group_CHILD', 'Sex_MALE', 'Hispanic Ethnicity_YES',
       'Veteran Status_YES', 'Mental Illness_YES',
       'Intellectual Disability_YES', 'Autism Spectrum_YES',
       'Other Developmental Disability_YES', 'Alcohol Related Disorder_YES',
       'Drug Substance Disorder_YES',
       ...
       'ZIP3=142', 'ZIP3=143', 'ZIP3=144', 'ZIP3=145', 'ZIP3=146', 'ZIP3=147',
       'ZIP3=148', 'ZIP3=149', 'ZIP3=888', 'ZIP3=999'],
      dtype='object', length=161)

In [3]:
#load the dataset:
df = pd.read_csv('./data/mental_illness.csv',index_col=0)

df.head()

,Age Group_CHILD,Sex_MALE,Hispanic Ethnicity_YES,Veteran Status_YES,Mental Illness_YES,Intellectual Disability_YES,Autism Spectrum_YES,Other Developmental Disability_YES,Alcohol Related Disorder_YES,Drug Substance Disorder_YES,...,ZIP3=142,ZIP3=143,ZIP3=144,ZIP3=145,ZIP3=146,ZIP3=147,ZIP3=148,ZIP3=149,ZIP3=888,ZIP3=999
0,0,1,1,0,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,1,0,0,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
#set the labels
labels = ['Mental Illness_YES',
 'Serious Mental Illness_YES',
 'Other Developmental Disability_YES',
 'Alcohol Related Disorder_YES',
 'Drug Substance Disorder_YES']

df.groupby(labels).size().reset_index().sort_values(by=0,ascending=False).reset_index(drop=True)

,Mental Illness_YES,Serious Mental Illness_YES,Other Developmental Disability_YES,Alcohol Related Disorder_YES,Drug Substance Disorder_YES,0
0,1,1,0,0,0,67248
1,1,0,0,0,0,12889
2,1,1,0,0,1,6837
3,1,1,0,1,1,5756
4,1,1,0,1,0,3116
5,0,0,0,0,0,2574
6,1,1,1,0,0,2247
7,0,1,0,0,0,1674
8,1,0,0,0,1,702
9,1,0,0,1,0,402


### Test some black boxes:

Split in training and test set

In [5]:
X = df.drop(labels,1)
y = df[labels]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [6]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import dummy

names = ["MLPClassifier",\
         "RandomForestClassifier",\
         "OneVsRestClassifier_SVM",\
         "OneVsRestClassifier_Adaboost",\
         "Dummy Stratified"]

classifiers = [MLPClassifier(),\
              RandomForestClassifier(),\
              OneVsRestClassifier(SVC(gamma='auto')),\
              OneVsRestClassifier(AdaBoostClassifier(n_estimators=1000)),\
              dummy.DummyClassifier(strategy='stratified')]

In [8]:
tabella_metriche_df.style.apply(highlight_max)

,accuracy score,1-hamming_loss,precision score sample,recall score sample,F1-score sample,precision score macro,recall score macro,F1-score macro,precision score micro,recall score micro,F1-score micro
MLPClassifier,0.606,0.898,0.88,0.882,0.859,0.593,0.508,0.529,0.876,0.875,0.875
RandomForestClassifier,0.612,0.9,0.893,0.87,0.859,0.581,0.438,0.456,0.896,0.856,0.876
OneVsRestClassifier_SVM,0.638,0.905,0.892,0.896,0.873,0.464,0.41,0.394,0.891,0.875,0.883
OneVsRestClassifier_Adaboost,0.633,0.905,0.888,0.897,0.872,0.652,0.467,0.486,0.885,0.883,0.884
Dummy Stratified,0.41,0.835,0.823,0.816,0.784,0.414,0.414,0.414,0.799,0.799,0.799


So now we have the black box

In [7]:
bb = joblib.load('./black_boxes/OneVsRestClassifier_Adaboost_mental.pkl')

And now... **MARLENA**

https://github.com/CeciPani/MARLENA

<center><img style="width: 60%; height: 60%" src="./data/gitignoreDATA/presentazione_lipari_MARLENA/2_outcome_explanation.png"></center>

1. It generates a synthetic neighborhood around the instance to be explained using a strategy suitable for multi-label decisions.
2. It learns a decision tree (DT) on such neighborhood.
3. It derives from the DT a decision rule that explains the black box decision.

### How MARLENA builds the synthetic neighborhood

Suppose we want to explain this instance

<center><img style="width: 40%; height: 40%" src="./data/gitignoreDATA/presentazione_lipari_MARLENA/dataset_sample.png"></center>

### MARLENA starts from a the feature distribution of the closest neighbors of the instance

<center><img style="width: 40%; height: 40%" src="./data/gitignoreDATA/presentazione_lipari_MARLENA/mixed_real_neigh.png"></center>

### Selecting the real neighbors: feature space

A fraction $\alpha$ of these neighbors are the closest in the feature space according to a distance function $d_f(x,\hat{x})$
<center><img style="width: 40%; height: 40%" src="./data/gitignoreDATA/presentazione_lipari_MARLENA/mixed_features_real_neigh.png"></center> 

### Selecting the real neighbors: label space

A fraction $1-\alpha$ of these neighbors are the closest in the **label** space according to a distance function $d_l(b(x),b(\hat{x}))$

<center><img style="width: 40%; height: 40%" src="./data/gitignoreDATA/presentazione_lipari_MARLENA/mixed_labels_real_neigh.png"></center>

### MARLENA then create the synthetic neighbors perturbing the real ones.

From the distribution of the features of these real neighbors MARLENA creates the synthetic ones. Then the black box is used to label them to generate a new **ground truth** for the interpretable classifier (DT).

<center><img style="width: 50%; height: 50%" src="./data/gitignoreDATA/presentazione_lipari_MARLENA/labeling.png"></center>

In [8]:
from marlena.marlena import MARLENA

Istantiate the MARLENA object

In [9]:
m1 = MARLENA(neigh_type='mixed')

In [10]:
#select one instance whose multi-label decision we want to explain:
instance_to_be_explained = df.drop(labels,1).loc[6]

# black box decision:
print(f'black box prediction: {bb.predict(np.array(instance_to_be_explained.values).reshape(1, -1))}')

#remember that:
labels

black box prediction: [[1 1 0 1 0]]


['Mental Illness_YES',
 'Serious Mental Illness_YES',
 'Other Developmental Disability_YES',
 'Alcohol Related Disorder_YES',
 'Drug Substance Disorder_YES']

To extract the explanation, MARLENA needs:

In [11]:
# the black box:
bb = joblib.load('./black_boxes/OneVsRestClassifier_Adaboost_mental.pkl')

In [12]:
# the instance to be explained:
i2e = instance_to_be_explained

In [13]:
#the name of the classes:
labels_name = labels
#which variables are categorical and which are numerical:
numerical_vars = []
categorical_vars = df.drop(labels,1).columns.values

In [14]:
# a set of instances representative of the dataset that was used to train the black box 
X2E = df.drop(labels_name,1)

#### MARLENA hyperparameters:
* **k**: number of real neighbors to use to generate the synthetic ones 
* **alpha**: fraction of neighbors to be sampled from neighbors in the feature space
* **k_synth**: number of synthetic neighbots

In [15]:
k = 50
k_synt = 1000
alpha = 0.7

In [16]:
labels_name = ['Mental Illness',
 'Serious Mental Illness',
 'Other Developmental Disability',
 'Alcohol Related Disorder',
 'Drug Substance Disorder']

### MARLENA "extract_explanation" method:

In [17]:
rule, instance_imporant_feat, fidelity, hit, DT = m1.extract_explanation(i2e,\
                                                                         X2E,\
                                                                         bb,\
                                                                         numerical_vars,\
                                                                         categorical_vars,\
                                                                         labels_name,\
                                                                         k=k,\
                                                                         size=k_synt,\
                                                                         alpha=alpha)

MARLENA-mixed
decision rule: {Neurological Condition_YES = False,
 Age Group_CHILD = False,
 Intellectual Disability_YES = False,
 Smokes_YES = True,
 ZIP3=888 = False,
 Program Category=INPATIENT = True,
 Alzheimer or Dementia_YES = True,
 Employment Status=NOT IN LABOR FORCE:UNEMPLOYED AND NOT LOOKING FOR WORK = True,
 ZIP3=103 = False} -> ['Mental Illness' 'Serious Mental Illness' 'Alcohol Related Disorder']
rule length: 9
black-box decision: [[1 1 0 1 0]]
fidelity of DT: 0.9706728004600345
hit: 1.0


### Marlena output:

In [18]:
#decision rule
print(rule)

{Neurological Condition_YES = False,
 Age Group_CHILD = False,
 Intellectual Disability_YES = False,
 Smokes_YES = True,
 ZIP3=888 = False,
 Program Category=INPATIENT = True,
 Alzheimer or Dementia_YES = True,
 Employment Status=NOT IN LABOR FORCE:UNEMPLOYED AND NOT LOOKING FOR WORK = True,
 ZIP3=103 = False} -> ['Mental Illness' 'Serious Mental Illness' 'Alcohol Related Disorder']


* Program Category "INPATIENT": inpatient services provide stabilization and intensive treatment and rehabilitation with 24-hour care in a controlled environment.
* Since the 3-digit zip code is not 888 the client was NOT homeless at the time of the survey

In [19]:
#fidelity
print(fidelity)

0.9706728004600345


$$fidelity(Y,\hat{Y})\in\left[0,1\right]$$

The **fidelity** compares the decisions of the DT to those of the black-box on the synthetic neighborhood. It is measured using the *F1-score*.

In [20]:
#hit
print(hit)

1.0


$$hit(y,\hat{y})) = 1 − hamming(y, ŷ) \in \left[0,1\right]$$

The **hit** compares the predictions of the DT and the black-box on the instance *x* under analysis. It is mesured usinf the *simple match similarity* 

# Thank you